In [67]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from typing import Tuple, Optional
from pathlib import Path
import datetime
import io
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import gpflow
from gpflow.config import default_float
from gpflow.utilities import set_trainable, print_summary
import warnings
warnings.filterwarnings('ignore')


In [59]:
batch_size = 128
num_classes = 10
epochs = 8

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

nn_model = Sequential()
nn_model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
nn_model.add(Conv2D(64, (3, 3), activation='relu'))
nn_model.add(MaxPooling2D(pool_size=(2, 2)))
nn_model.add(Dropout(0.25))
nn_model.add(Flatten())
nn_model.add(Dense(128, activation='relu'))
nn_model.add(Dropout(0.5))

nn_model.add(Dense(num_classes, activation='softmax'))

nn_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

nn_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = nn_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/8
60000/60000 [==============================] - 64s 1ms/step - loss: 0.2724 - accuracy: 0.9163 - val_loss: 0.0585 - val_accuracy: 0.9812
Epoch 2/8
60000/60000 [==============================] - 62s 1ms/step - loss: 0.0904 - accuracy: 0.9739 - val_loss: 0.0398 - val_accuracy: 0.9868
Epoch 3/8
60000/60000 [==============================] - 63s 1ms/step - loss: 0.0676 - accuracy: 0.9801 - val_loss: 0.0374 - val_accuracy: 0.9882
Epoch 4/8
60000/60000 [==============================] - 64s 1ms/step - loss: 0.0579 - accuracy: 0.9828 - val_loss: 0.0365 - val_accuracy: 0.9877
Epoch 5/8
60000/60000 [==============================] - 66s 1ms/step - loss: 0.0487 - accuracy: 0.9853 - val_loss: 0.0327 - val_accuracy: 0.9890
Epoch 6/8
60000/60000 [==============================] - 63s 1ms/step - loss: 0.0435 - accuracy: 0.9870 - val_loss: 0.0325 - val_accuracy: 0.9901
E

In [61]:
# No extract one intermediate layer output:
get_layer_output = K.function([nn_model.layers[0].input, K.learning_phase()],
                                  [nn_model.layers[6].output])
# output in train mode = 1
layer_output_train = get_layer_output([x_train, 1])[0]

# output in test mode = 0
layer_output_test = get_layer_output([x_test, 0])[0]

In [1]:
print(layer_output_train.shape)
# We now want to build a GP classsifier that takes this as input and maps it to the mnist y_train
#Set up random seeds and default float for gpflow tensors:
gpflow.config.set_default_float(np.float64)
np.random.seed(0)
tf.random.set_seed(0)

NameError: name 'layer_output_train' is not defined

In [78]:
train_dataset = tf.data.Dataset.from_tensor_slices((layer_output_train.astype('float64'), y_train.astype('float64')))
test_dataset = tf.data.Dataset.from_tensor_slices((layer_output_test.astype('float64'), y_test.astype('float64')))

num_train_data = 60000
batch_size = 32
num_features = 128
prefetch_size = num_train_data // 2
shuffle_buffer_size = num_train_data // 2
num_batches_per_epoch = num_train_data // batch_size

original_train_dataset = train_dataset
train_dataset = train_dataset.repeat()\
                    .prefetch(prefetch_size)\
                    .shuffle(buffer_size=shuffle_buffer_size)\
                    .batch(batch_size)

print(f"prefetch_size={prefetch_size}")
print(f"shuffle_buffer_size={shuffle_buffer_size}")
print(f"num_batches_per_epoch={num_batches_per_epoch}")

prefetch_size=30000
shuffle_buffer_size=30000
num_batches_per_epoch=1875


In [86]:
# GP model:
kernel = gpflow.kernels.RBF(variance=2.)
likelihood = gpflow.likelihoods.Gaussian()
# A bit unsure here
inducing_variable = np.linspace(0, 10, num_features).reshape(-1, 1)

N = 60000 # Training data points
M = 500  # Number of inducing locations
Z = layer_output_train[:M, :].copy()  # Initialize inducing locations to the first M inputs in the dataset

gp_model = gpflow.models.SVGP(kernel, likelihood, Z, num_data=N)

In [80]:
set_trainable(likelihood, False)
set_trainable(kernel.variance, False)
set_trainable(likelihood, True)
set_trainable(kernel.variance, True)
print_summary(gp_model)  

name                      class      transform       trainable    shape          dtype    value
------------------------  ---------  --------------  -----------  -------------  -------  ----------------
SVGP.kernel.variance      Parameter  Softplus        True         ()             float64  2.0
SVGP.kernel.lengthscale   Parameter  Softplus        True         ()             float64  1.0
SVGP.likelihood.variance  Parameter  Softplus        True         ()             float64  1.0
SVGP.inducing_variable.Z  Parameter                  True         (128, 1)       float64  [[0....
SVGP.q_mu                 Parameter                  True         (128, 1)       float64  [[0....
SVGP.q_sqrt               Parameter  FillTriangular  True         (1, 128, 128)  float64  [[[1., 0., 0....


In [81]:
# Let's start training
gp_optimizer = tf.optimizers.Adam()
def optimization_step(model: gpflow.models.SVGP, batch: Tuple[tf.Tensor, tf.Tensor]):
    with tf.GradientTape(watch_accessed_variables=False) as tape:
        tape.watch(model.trainable_variables)
        obj = - model.elbo(*batch)
        grads = tape.gradient(obj, model.trainable_variables)
    gp_optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [82]:
def simple_training_loop(model: gpflow.models.SVGP, epochs: int = 1, logging_epoch_freq: int = 10):
    batches = iter(train_dataset)
    tf_optimization_step = tf.function(optimization_step, autograph=False)
    for epoch in range(epochs):
        for _ in range(num_batches_per_epoch):
            tf_optimization_step(model, next(batches))

        epoch_id = epoch + 1
        if epoch_id % logging_epoch_freq == 0:
            tf.print(f"Epoch {epoch_id}: ELBO (train) {model.elbo(data)}")

In [83]:
simple_training_loop(gp_model, epochs=10, logging_epoch_freq=2)

ValueError: Dimensions must be equal, but are 1 and 128 for 'Tensordot/MatMul' (op: 'MatMul') with input shapes: [128,1], [128,32].

ModuleNotFoundError: No module named 'multiclass_classification'